In [1]:
### Import dependencies
# Dependencies for data manipulation
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import *

# Dependencies for Databases
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
from sqlalchemy import create_engine

# Dependencies for APIs
from bs4 import BeautifulSoup
import requests
import json

# Dependencies for Webscraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

# Internal imports
from db import get_df

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)


In [120]:
# Get all players' name, team name, position
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=players"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    data.append(rows)
scrape1 = pd.DataFrame(data)
scrape1.columns=['PlayerID','Name', 'Position', 'Team']

# Get Shark Ranks
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=playerRanks"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
sharkRanks = soup.find_all('player')
for i in range(len(sharkRanks)):
    rows = [sharkRanks[i].get("id"), sharkRanks[i].get("rank")]
    data.append(rows)
shark_df = pd.DataFrame(data)
shark_df.columns=['PlayerID','SharkRank']
shark_df['SharkRank'] = shark_df['SharkRank'].astype('int32')

# Get ADP
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=adp"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("averagePick")]
    data.append(rows)
adp_df = pd.DataFrame(data)
adp_df.columns=['PlayerID','ADP']
adp_df['ADP'] = adp_df['ADP'].astype('float32')

# Get player ages
# Get any player dobs who are already in the db
player_dobs = get_df('player_dobs')
# Check for any players whose ages are not already in the db
to_query_age = scrape1[~scrape1['PlayerID'].isin(player_dobs['PlayerID'])]
if len(to_query_age)>0:
    # Break player list into chunks small enough for the API server
    n = 50  #chunk row size
    list_df = [to_query_age.PlayerID[i:i+n] for i in range(0,to_query_age.PlayerID.shape[0],n)]

    for i in range(len(list_df)):
        idList = ",".join(list_df[i])

        # Get playerProfiles
        urlString = f"https://api.myfantasyleague.com/2022/export?TYPE=playerProfile&P={idList}"
        response = requests.get(urlString)
        soup = BeautifulSoup(response.content,'xml')
        data = []
        profiles = soup.find_all('playerProfile')
        players = soup.find_all('player')
        for i in range(len(profiles)):
            rows = [profiles[i].get("id"), players[i].get("dob")]
            data.append(rows)
        data_df = pd.DataFrame(data)
        age = pd.DataFrame(columns=['PlayerID', 'DOB'])
        age['PlayerID'] = data_df[0]
        age['DOB'] = data_df[1]
        player_dobs = player_dobs.append(age)

# Convert string to datetime
player_dobs['DOB'] = pd.to_datetime(player_dobs['DOB'])
# Convert DOB to Age
today = date.today()
def age(born):
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
player_dobs['Age'] = player_dobs['DOB'].apply(age)

# Merge all dfs from MyFantasyLeague API
scrape1 = scrape1.merge(player_dobs, on='PlayerID', how='left')
scrape1 = scrape1.drop(columns='DOB')
scrape1 = scrape1.merge(shark_df, on='PlayerID', how='left').merge(adp_df, on='PlayerID', how='left')
scrape1['SharkRank'].fillna(3000, inplace=True)
scrape1['ADP'].fillna(3000, inplace=True)
scrape1 = scrape1.sort_values(by=['SharkRank'])
scrape1.reset_index(inplace=True, drop=True)  

### Clean MFL data
## Select only relevant positions
scrape1 = scrape1.loc[scrape1['Position'].isin(['QB', 'WR', 'RB', 'TE', 'PK', 'Def'])]
scrape1 = scrape1.reset_index(drop=True)
## Clean Name column
to_join = scrape1['Name'].str.split(", ", n=1, expand=True)
to_join.columns = ['lname', 'fname']
to_join['Name'] = to_join['fname'] + " " + to_join['lname']
scrape1['Name'] = to_join['Name']
# Change name to Title Case
scrape1['Name'] = scrape1['Name'].str.upper()
# Drop name punctuation
scrape1['Name'] = scrape1['Name'].str.replace(".", "")
scrape1['Name'] = scrape1['Name'].str.replace(",", "")
scrape1['Name'] = scrape1['Name'].str.replace("'", "")
## Clean position column
scrape1['Position'] = scrape1['Position'].replace('Def', 'DF')
# Clean Team column
scrape1['Team'] = scrape1['Team'].replace('FA*', 'FA')
## Change column names
scrape1.columns = ['id_mfl', 'player', 'pos_mfl', 'team', 'age', 'sharkRank', 'adp']
scrape1

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:97: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,id_mfl,player,pos_mfl,team,age,sharkRank,adp
0,14802,JONATHAN TAYLOR,RB,IND,23.0,1.0,1.37
1,13404,AUSTIN EKELER,RB,LAC,27.0,2.0,4.57
2,13130,CHRISTIAN MCCAFFREY,RB,CAR,26.0,3.0,3.47
3,12626,DERRICK HENRY,RB,TEN,28.0,4.0,5.32
4,13164,COOPER KUPP,WR,LAR,29.0,5.0,5.45
...,...,...,...,...,...,...,...
1095,13158,DEDE WESTBROOK,WR,FA,28.0,3000.0,3000.00
1096,13155,JOHN ROSS,WR,FA,26.0,3000.0,3000.00
1097,13144,ELIJAH MCGUIRE,RB,FA,28.0,3000.0,3000.00
1098,13143,JEREMY MCNICHOLS,RB,PIT,26.0,3000.0,3000.00


In [121]:
### scrape2 posRanks
# Set Selenium settings
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
# scrape2 web for stats
url = f"https://www.ourlads.com/nfldepthcharts/depthcharts.aspx"
PATH = "/Applications/chromedriver"
driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
wait = WebDriverWait(driver, 20)
driver.get(url)
wait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='ctl00_phContent_gvChart']")))
driver.execute_script("window.stop();")

scrape2 = pd.read_html(driver.find_element(By.XPATH, value="//table[@id='ctl00_phContent_gvChart']").get_attribute("outerHTML"))
scrape2 = scrape2[0]

In [122]:
### Clean scrape2d data
scrape2 = scrape2[['Team', 'Pos', 'Player 1', 'Player 2','Player 3', 'Player 4', 'Player 5']]

# Transform columns into rows
scrape21 = scrape2[['Team', 'Pos', 'Player 1']]
scrape21 = scrape21.rename(columns={'Player 1':'Player'})
scrape21['posRank'] = "1"

scrape22 = scrape2[['Team', 'Pos', 'Player 2']]
scrape22 = scrape22.rename(columns={'Player 2':'Player'})
scrape22['posRank'] = "2"

scrape23 = scrape2[['Team', 'Pos', 'Player 3']]
scrape23 = scrape23.rename(columns={'Player 3':'Player'})
scrape23['posRank'] = "3"

scrape24 = scrape2[['Team', 'Pos', 'Player 4']]
scrape24 = scrape24.rename(columns={'Player 4':'Player'})
scrape24['posRank'] = "4"

scrape25 = scrape2[['Team', 'Pos', 'Player 5']]
scrape25 = scrape25.rename(columns={'Player 5':'Player'})
scrape25['posRank'] = "5"

scrape2_complete = pd.concat([scrape21, scrape22, scrape23, scrape24, scrape25], axis=0, ignore_index=True)

# Clean Position column
# Select only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
scrape2_final = scrape2_complete.loc[scrape2_complete['Pos'].isin(posList)]
# Convert WR roles to "WR"
scrape2_final['Pos'].replace(["LWR", "RWR", "SWR"], "WR", inplace=True)
scrape2_final['posRank'] = scrape2_final['Pos'] + scrape2_final['posRank']
scrape2_final = scrape2_final.reset_index(drop=True)
scrape2_final.dropna(inplace=True)
scrape2_final.drop_duplicates(subset=['Player', 'Team', 'Pos'], inplace=True)

# Create columns for KRs and PRs
krs = scrape2_final.loc[scrape2_final.Pos=='KR']
krs = krs.drop(columns=['Pos'])
krs.columns = ['Team', 'Player', 'KR']
prs = scrape2_final.loc[scrape2_final.Pos=='PR']
prs = prs.drop(columns=['Pos'])
prs.columns = ['Team', 'Player', 'PR']
# Join pr and pk scrape2s back onto main ourlads scrape2
scrape2_final = scrape2_final.merge(krs, how='left', on=['Player', 'Team']).merge(prs, how='left', on=['Player', 'Team'])
scrape2_final['KR'].fillna("NO", inplace=True)
scrape2_final['PR'].fillna("NO", inplace=True)

# Clean name column
names = scrape2_final['Player'].str.split(" ", n=2, expand=True)
names.columns = ['a', 'b', 'c']
names['a'] = names['a'].str.replace(",", "")
scrape2_final['Player'] = names['b'] + " " + names['a']
# Change to Upper Case
scrape2_final['Player'] = scrape2_final['Player'].str.upper()
# Drop punctuation
scrape2_final['Player'] = scrape2_final['Player'].str.replace(".", "")
scrape2_final['Player'] = scrape2_final['Player'].str.replace(",", "")
scrape2_final['Player'] = scrape2_final['Player'].str.replace("'", "")

# Change column names and order
scrape2_final = scrape2_final[['Player', 'Pos', 'Team', 'posRank', 'KR', 'PR']]
scrape2_final.columns = ['player', 'pos_ol', 'team', 'posRank', 'KR', 'PR']

# Remove separate rows for PRs and KRs
scrape2_final = scrape2_final.loc[(scrape2_final.pos_ol!="KR")]
scrape2_final = scrape2_final.loc[(scrape2_final.pos_ol!="PR")]

# Drop position column
scrape2_final.drop(columns=['pos_ol'], inplace=True)
scrape2_final

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions

,player,team,posRank,KR,PR
0,AJ GREEN,ARZ,WR1,NO,NO
1,MARQUISE BROWN,ARZ,WR1,NO,NO
2,RONDALE MOORE,ARZ,WR1,KR1,PR1
3,ZACH ERTZ,ARZ,TE1,NO,NO
4,KYLER MURRAY,ARZ,QB1,NO,NO
...,...,...,...,...,...
1102,JULIUS CHESTNUT,TEN,RB5,NO,NO
1103,KENDALL BLANTON,WAS,TE5,NO,NO
1104,JAKE HAUSMANN,WAS,TE5,NO,NO
1105,JONATHAN WILLIAMS,WAS,RB5,NO,NO


In [157]:
### Merge MyFantasyLeague data with scrape2d data
player_df = scrape1.merge(scrape2_final, how='left', on=['player', 'team'])
## Clean merged df
player_df.loc[player_df['pos_mfl']=='DF', 'posRank'] = "DF1"
player_df['KR'].fillna("NO", inplace=True)
player_df['PR'].fillna("NO", inplace=True)
## Clean posRanks
player_df['posRank'] = player_df['posRank'].map({
    'RES1':'RES',
    'RES2':'RES',
    'RES3':'RES',
    'RES4':'RES',
    'RES5':'RES',
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB3',
    'QB5':'QB3', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB3', 
    'RB5':'RB3',
    'WR1': 'WR1', 
    'WR2': 'WR2', 
    'WR3': 'WR3', 
    'WR4': 'WR3', 
    'WR5': 'WR3', 
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE3', 
    'TE5':'TE3', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })
# Create "RES/NO" column
player_df['RES'] = "NO"
player_df.loc[player_df['posRank']=="RES", 'RES'] = "RES"
player_df.loc[player_df.posRank.isna(), 'posRank'] = player_df.loc[player_df.posRank.isna(), 'pos_mfl'] + "3"
player_df.loc[player_df.posRank=="RES", 'posRank'] = player_df.loc[player_df.posRank=="RES", 'pos_mfl'] + "3"
# Specify all players are in current season
player_df['season'] = 2022
player_df

,id_mfl,player,pos_mfl,team,age,sharkRank,adp,posRank,KR,PR,RES,season
0,14802,JONATHAN TAYLOR,RB,IND,23.0,1.0,1.37,RB1,NO,NO,NO,2022
1,13404,AUSTIN EKELER,RB,LAC,27.0,2.0,4.57,RB1,NO,NO,NO,2022
2,13130,CHRISTIAN MCCAFFREY,RB,CAR,26.0,3.0,3.47,RB1,NO,NO,NO,2022
3,12626,DERRICK HENRY,RB,TEN,28.0,4.0,5.32,RB1,NO,NO,NO,2022
4,13164,COOPER KUPP,WR,LAR,29.0,5.0,5.45,WR1,NO,PR2,NO,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
1095,13158,DEDE WESTBROOK,WR,FA,28.0,3000.0,3000.00,WR3,NO,NO,NO,2022
1096,13155,JOHN ROSS,WR,FA,26.0,3000.0,3000.00,WR3,NO,NO,NO,2022
1097,13144,ELIJAH MCGUIRE,RB,FA,28.0,3000.0,3000.00,RB3,NO,NO,NO,2022
1098,13143,JEREMY MCNICHOLS,RB,PIT,26.0,3000.0,3000.00,RB3,NO,NO,RES,2022


In [158]:
### Get historical data
prior1 = get_df('prior1')
prior2 = get_df('prior2')


In [159]:
# Create current_df
# This will mean scraping the ff db site weekly
curr = pd.DataFrame(player_df['player'])
colList = ['gamesPlayed',
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY','rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM','FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk','defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst','defYdsAgainst'
]
cols = pd.DataFrame(columns=colList)
curr = curr.merge(cols, how='left', left_index=True, right_index=True)
curr.fillna(0, inplace=True)

# Rename all columns in curr
colList = [(x + "_curr") for x in list(curr.columns)]
curr.columns = colList
curr = curr.rename(columns={
       'player_curr':'player',
       })
curr

,player,gamesPlayed_curr,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,defSack_curr,defI_curr,defSaf_curr,defFum_curr,defBlk_curr,defT_curr,defPtsAgainst_curr,defPassYAgainst_curr,defRushYAgainst_curr,defYdsAgainst_curr
0,JONATHAN TAYLOR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUSTIN EKELER,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHRISTIAN MCCAFFREY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DERRICK HENRY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,COOPER KUPP,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,DEDE WESTBROOK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1096,JOHN ROSS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1097,ELIJAH MCGUIRE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1098,JEREMY MCNICHOLS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
# Merge playerdf, currentdf, prior1, and prior2
player_df = player_df.merge(curr, how='left', on='player').merge(prior1, how='left', on='player').merge(prior2, how='left', on='player')
# Fill data for players who do not have prior data
player_df.fillna(0, inplace=True)

In [161]:
player_df.drop_duplicates(subset=['player', 'pos_mfl'], inplace=True)

In [162]:
# Get schedule
schedule = get_df('schedule')
# Merge in opponents
player_df = player_df.merge(schedule, how='left', on='team')
print(len(player_df))

14428


In [163]:
# Rename position column in player_df
player_df.rename(columns={'pos_mfl':'pos'}, inplace=True)

In [164]:
# Get opponent historical data
# select only defenses
allDef = player_df.loc[player_df['pos']=='DF']
allDef

,id_mfl,player,pos,team,age,sharkRank,adp,posRank,KR,PR,...,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2,week,opponent
2703,0528,LOS ANGELES RAMS,DF,LAR,0.0,310.0,112.919998,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,3.0,ARI
2704,0528,LOS ANGELES RAMS,DF,LAR,0.0,310.0,112.919998,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,4.0,SFO
2705,0528,LOS ANGELES RAMS,DF,LAR,0.0,310.0,112.919998,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,9.0,TBB
2706,0528,LOS ANGELES RAMS,DF,LAR,0.0,310.0,112.919998,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,11.0,NOS
2707,0528,LOS ANGELES RAMS,DF,LAR,0.0,310.0,112.919998,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,12.0,KCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6067,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,1162.0,171.039993,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,10.0,IND
6068,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,1162.0,171.039993,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,13.0,LAC
6069,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,1162.0,171.039993,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,15.0,NEP
6070,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,1162.0,171.039993,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,17.0,SFO


In [165]:

# Get current defensive scores
currDef = allDef.copy()
# Select only relevant columns
currDef = currDef[['team', 'week',
       'defSack_curr', 'defI_curr',
       'defSaf_curr', 'defFum_curr', 'defBlk_curr',
       'defT_curr', 'defPtsAgainst_curr', 'defPassYAgainst_curr',
       'defRushYAgainst_curr', 'defYdsAgainst_curr']]

# Get prior defensive scores
priorDef = allDef.copy()
# Select only relevant columns
priorDef = priorDef[['team', 'week',
       'defSack_prior1', 'defI_prior1',
       'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1',
       'defT_prior1', 'defPtsAgainst_prior1', 'defPassYAgainst_prior1',
       'defRushYAgainst_prior1', 'defYdsAgainst_prior1']]
# Merge the two defensive dfs
allDef = currDef.merge(priorDef, how='left', on=['team', 'week'])

# Rename all columns in allDef
colList = [(x + "_opp") for x in list(allDef.columns)]
allDef.columns = colList
allDef = allDef.rename(columns={
       'team_opp':'opponent',
       'week_opp':'week'
       })

In [166]:
# Connect opponents to defenses
player_df = player_df.merge(allDef, how='left', on=['opponent', 'week'])
print(len(player_df))

14428


In [167]:
str(list(player_df.columns))

"['id_mfl', 'player', 'pos', 'team', 'age', 'sharkRank', 'adp', 'posRank', 'KR', 'PR', 'RES', 'season', 'gamesPlayed_curr', 'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 'rushA_curr', 'rushY_curr', 'rushT_curr', 'rush2_curr', 'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 'FG50_curr', 'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 'defPtsAgainst_curr', 'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 'gamesPlayed_prior1', 'passA_prior1', 'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 'rush2_prior1', 'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 'FGM_prior1', 'FG50_prior1', 'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 'defPt

In [168]:

player_df = player_df[[
    'season',
    'week',
    'team',
    'player',
    'age',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent',
    'passA_curr',
    'passC_curr',
    'passY_curr',
    'passT_curr',
    'passI_curr',
    'pass2_curr',
    'rushA_curr',
    'rushY_curr',
    'rushT_curr',
    'rush2_curr',
    'recC_curr',
    'recY_curr',
    'recT_curr',
    'rec2_curr',
    'fum_curr',
    'XPA_curr',
    'XPM_curr',
    'FGA_curr',
    'FGM_curr',
    'FG50_curr',
    'defSack_curr',
    'defI_curr',
    'defSaf_curr',
    'defFum_curr',
    'defBlk_curr',
    'defT_curr',
    'defPtsAgainst_curr',
    'defPassYAgainst_curr',
    'defRushYAgainst_curr',
    'defYdsAgainst_curr',
    'gamesPlayed_curr',
    'gamesPlayed_prior1',
    'passA_prior1',
    'passC_prior1',
    'passY_prior1',
    'passT_prior1',
    'passI_prior1',
    'pass2_prior1',
    'rushA_prior1',
    'rushY_prior1',
    'rushT_prior1',
    'rush2_prior1',
    'recC_prior1',
    'recY_prior1',
    'recT_prior1',
    'rec2_prior1',
    'fum_prior1',
    'XPA_prior1',
    'XPM_prior1',
    'FGA_prior1',
    'FGM_prior1',
    'FG50_prior1',
    'defSack_prior1',
    'defI_prior1',
    'defSaf_prior1',
    'defFum_prior1',
    'defBlk_prior1',
    'defT_prior1',
    'defPtsAgainst_prior1',
    'defPassYAgainst_prior1',
    'defRushYAgainst_prior1',
    'defYdsAgainst_prior1',
    'gamesPlayed_prior2',
    'passA_prior2',
    'passC_prior2',
    'passY_prior2',
    'passT_prior2',
    'passI_prior2',
    'pass2_prior2',
    'rushA_prior2',
    'rushY_prior2',
    'rushT_prior2',
    'rush2_prior2',
    'recC_prior2',
    'recY_prior2',
    'recT_prior2',
    'rec2_prior2',
    'fum_prior2',
    'XPA_prior2',
    'XPM_prior2',
    'FGA_prior2',
    'FGM_prior2',
    'FG50_prior2',
    'defSack_prior2',
    'defI_prior2',
    'defSaf_prior2',
    'defFum_prior2',
    'defBlk_prior2',
    'defT_prior2',
    'defPtsAgainst_prior2',
    'defPassYAgainst_prior2',
    'defRushYAgainst_prior2',
    'defYdsAgainst_prior2',
    'defSack_curr_opp',
    'defI_curr_opp',
    'defSaf_curr_opp',
    'defFum_curr_opp',
    'defBlk_curr_opp',
    'defT_curr_opp',
    'defPtsAgainst_curr_opp',
    'defPassYAgainst_curr_opp',
    'defRushYAgainst_curr_opp',
    'defYdsAgainst_curr_opp',
    'defSack_prior1_opp',
    'defI_prior1_opp',
    'defSaf_prior1_opp',
    'defFum_prior1_opp',
    'defBlk_prior1_opp',
    'defT_prior1_opp',
    'defPtsAgainst_prior1_opp',
    'defPassYAgainst_prior1_opp',
    'defRushYAgainst_prior1_opp',
    'defYdsAgainst_prior1_opp']]

,gamesPlayed_curr,gamesPlayed_prior1,gamesPlayed_prior2
0,0,16.0,31.0
1,0,16.0,31.0
2,0,16.0,31.0
3,0,16.0,31.0
4,0,16.0,31.0
...,...,...,...
14423,0,13.0,27.0
14424,0,13.0,27.0
14425,0,13.0,27.0
14426,0,13.0,27.0
